In [1]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow_datasets import ImageFolder
import numpy as np


In [2]:
#upload kaggle.json file
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (3).json


{'kaggle.json': b'{"username":"timurzakhoda","key":"ababcf6531560afb23467036bb78d088"}'}

In [3]:
!ls -lga kaggle.json

-rw-r--r-- 1 root 68 Feb 25 10:47 kaggle.json


In [4]:
!pip install -q kaggle

In [5]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle datasets list

ref                                                       title                                         size  lastUpdated          downloadCount  
--------------------------------------------------------  -------------------------------------------  -----  -------------------  -------------  
michau96/restaurant-business-rankings-2020                Restaurant Business Rankings 2020             16KB  2021-01-30 14:20:45           3352  
gpreda/reddit-wallstreetsbets-posts                       Reddit WallStreetBets Posts                    8MB  2021-02-25 09:03:43           1747  
yorkehead/stock-market-subreddits                         Stock Market Subreddits                        1MB  2021-01-29 13:53:50            344  
aagghh/crypto-telegram-groups                             Crypto telegram groups                       297MB  2021-02-02 09:58:25            202  
timoboz/superbowl-history-1967-2020                       Superbowl History 1967 - 2020                  2KB  2020-02-

In [7]:
!kaggle datasets list -s ASL

ref                                                          title                                               size  lastUpdated          downloadCount  
-----------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  
grassknoted/asl-alphabet                                     ASL Alphabet                                         1GB  2018-04-22 19:31:36          18592  
danrasband/asl-alphabet-test                                 ASL Alphabet Test                                   24MB  2018-08-01 04:02:18           1768  
mrgeislinger/asl-rgb-depth-fingerspelling-spelling-it-out    ASL Fingerspelling Images (RGB & Depth)              2GB  2019-12-17 02:52:11            627  
lucasvieirademiranda/aslalphabet                             asl-alphabet                                         2GB  2018-11-23 17:36:46            140  
datamunge/sign-language-mnist                                Sig

In [8]:
!kaggle datasets download -d grassknoted/asl-alphabet    

asl-alphabet.zip: Skipping, found more recently modified local copy (use --force to force download)


In [9]:
!unzip ./asl-alphabet.zip

Archive:  ./asl-alphabet.zip
replace asl_alphabet_test/asl_alphabet_test/A_test.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [10]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [11]:
@tf.function
def augment(x, y):
    image = tf.image.random_brightness(x, max_delta=0.05)
    image = tf.image.random_flip_left_right(x)
    return image, y

@tf.function
def normalize(x,y):
     return tf.cast(x, tf.float32) / 255.0, y

In [12]:
train_dataset= image_dataset_from_directory('./asl_alphabet_train/asl_alphabet_train/',
                                            shuffle=True,
                                            batch_size=128,
                                            image_size=(128,128),
                                            seed = 123,
                                            validation_split=0.1,
                                            subset='training',
                                           )

Found 87000 files belonging to 29 classes.
Using 78300 files for training.


In [13]:
val_dataset= image_dataset_from_directory('./asl_alphabet_train/asl_alphabet_train/',
                                            shuffle=True,
                                            batch_size=128,
                                            image_size=(128,128),
                                            seed = 123,
                                            validation_split=0.1,
                                            subset='validation',
                                           )

Found 87000 files belonging to 29 classes.
Using 8700 files for validation.


In [14]:
train_dataset = train_dataset.map(augment)
train_dataset = train_dataset.map(normalize)
val_dataset = val_dataset.map(normalize)


In [15]:
def my_model():
    input = keras.Input(shape= (128,128,3))
    x = layers.Conv2D(32,3)(input)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(64,3)(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(128,3)(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(256,3)(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(64,1)(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu')(x)
    outputs = layers.Dense(29, activation='softmax')(x)
    return keras.Model(input, outputs)

In [16]:
model = my_model()

In [17]:
model.compile(optimizer='adam', loss= tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

In [18]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 126, 126, 32)      896       
_________________________________________________________________
batch_normalization (BatchNo (None, 126, 126, 32)      128       
_________________________________________________________________
tf.nn.relu (TFOpLambda)      (None, 126, 126, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 61, 61, 64)        18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 61, 61, 64)        256   

In [19]:
class CustomCallback(keras.callbacks.EarlyStopping):
    def __init__(self, patience = 0):
        super(keras.callbacks.EarlyStopping, self).__init__()
        self.patience = patience
        self.best_weights = None
    def on_train_begin(self, logs=None):
        self.wait = 0
        self.stopped_epoch = 0
        self.best = np.Inf
    def on_epoch_end(self, epoch, logs=None):
        current = logs.get('loss')
        if np.less(current, self.best):
            self.best = current
            self.wait = 0
            self.best_weights = self.model.get_weights()
        else:
            self.wait +=1
            if self.wait >= self.patience:
                self.stopped_epoch = epoch
                self.model.stop_training = True
                print("Restored model weights from the end of the best epoch.")
                self.model.set_weights(self.best_weights)
        
    def on_train_end(self, logs=None):
        if self.stopped_epoch > 0:
            print("Epoch %05d: early stopping" % (self.stopped_epoch + 1))

In [20]:
%%time
model.fit(train_dataset, epochs=15, verbose=1,validation_data=val_dataset, callbacks=[CustomCallback()])

Epoch 1/15
612/612 [==============================] - 167s 269ms/step - loss: 1.5048 - accuracy: 0.5671 - val_loss: 0.3631 - val_accuracy: 0.8764
Epoch 2/15
612/612 [==============================] - 165s 268ms/step - loss: 0.0883 - accuracy: 0.9757 - val_loss: 0.5242 - val_accuracy: 0.8421
Epoch 3/15
612/612 [==============================] - 165s 269ms/step - loss: 0.0428 - accuracy: 0.9872 - val_loss: 0.1028 - val_accuracy: 0.9618
Epoch 4/15
612/612 [==============================] - 165s 268ms/step - loss: 0.0306 - accuracy: 0.9910 - val_loss: 0.2313 - val_accuracy: 0.9191
Restored model weights from the end of the best epoch.
Epoch 00004: early stopping
CPU times: user 6min 47s, sys: 3min 46s, total: 10min 34s
Wall time: 11min 2s


In [21]:
model.evaluate(val_dataset)

68/68 [==============================] - 12s 160ms/step - loss: 0.1028 - accuracy: 0.9618


[0.10279834270477295, 0.9618390798568726]

In [22]:
tf.saved_model.save(model,'/content/drive/MyDrive/')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/assets


In [23]:
saved_model_dir = '/content/drive/MyDrive/saved_model.pb'

In [24]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

In [25]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
model_lite = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpovdh0sy5/assets


In [26]:
with open('/content/drive/MyDrive/model.tflite', 'wb') as f:
    f.write(model_lite)